In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
import random, os
import cv2
from skimage.exposure import rescale_intensity
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from scipy import ndimage
from scipy import misc as sm
from os import listdir
from os.path import isfile, join
from skimage.draw import random_shapes
from zipfile import * 
sys.path.append('..')

In [2]:
random.seed();
n=5000          #number of images to load
img_scale = 128   #scale of the images
downscale_dim = 32
resize_output_orig= f"./noise_generation/artifacts_orig/"      #where to save images
resize_output_artifact= f"./noise_generation/artifacts_changed/"      #where to save changed images


In [12]:
def recursive_files(dir):
    """Used to recursively iterate through all files in specified dir"""
    files = []
    for r, d, f in os.walk(dir):
        for file in f:
                files.append(os.path.join(r, file))
    return files

def generate_data(amount, img_scale, dataset_dir, output_orig_dir, output_augmented_dir, process_function):
    """Generates new data using process_function. New data is loaded as img_scale x img_scale image,
         process_function is called on each of these images specified by dataset_dir. Output is split to
         output_orig_dir (original files) and output_augmented_dir (processed files).
    """
    rfiles = recursive_files(dataset_dir)

    for iteration in range(amount):
        rfile = rfiles[iteration]

        img = cv2.imread(rfile)
        img = resize(img, (img_scale, img_scale), anti_aliasing=True)

        img = np.float32(img)

        changed = process_function(img)

        r_parts = rfile.split('/')
        r_end = r_parts[len(r_parts)-1]
        no_suffix = r_end.split('.')[0]

        cv2.imwrite(f"{output_orig_dir}{no_suffix}.jpg", img*256)
        cv2.imwrite(f"{output_augmented_dir}{no_suffix}.jpg", changed*256)

        #print(f"[{iteration+1}/{amount}] {rfile} -> ", f"{output_orig_dir}{no_suffix}.jpg")
        #print(f"[{iteration+1}/{amount}] {rfile} -> ", f"{output_augmented_dir}{no_suffix}.jpg\n")

In [13]:
def add_artifact(img):
    size = random.randint(8,13)
    res, labels = random_shapes((size, size), max_shapes=1, shape='rectangle',multichannel=True)
    x_offset=np.random.randint(0, img.shape[0]-10)
    y_offset=np.random.randint(0, img.shape[0]-10)
    new_img = np.copy(img)
    new_img[y_offset:y_offset+res.shape[0], x_offset:x_offset+res.shape[1]] = res*0
    return new_img

def apply_noise(img):
    new_img = np.copy(img)
    noise_factor = 0.1
    new_img = new_img + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=new_img.shape)
    return new_img

def apply_artifacts(img):
    number_of_artifacts = random.randint(3, 7)
    for i in range(number_of_artifacts):
        img = add_artifact(img)
        
    return img

output_orig= f"./noise_generation/artifacts/artifacts_orig/"             #where to save images
output_artifact= f"./noise_generation/artifacts/artifacts_changed/"      #where to save changed images
dataset_path = "./noise_generation/artifacts/artifacts_orig/"
img_scale = 128   #scale of images

generate_data(5000, img_scale, dataset_path, output_orig, output_artifact, apply_artifacts)

ValueError: could not broadcast input array from shape (12,12,3) into shape (11,12,3)